In [ ]:
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import math
from sklearn.preprocessing import StandardScaler

In [ ]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
points=mpPose.PoseLandmark
pose = mpPose.Pose()

In [ ]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers

In [ ]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def calculateAngle(landmark1, landmark2, landmark3):
  

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [ ]:
cap=cv2.VideoCapture(0)
# size=cv2.resize(cap,(500,500))
# Initiate holistic model
with mpPose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1) as mp_pose:
        # Recolor Feed
    while cap.isOpened():
        ret,frame=cap.read()
        frame=cv2.flip(frame,1)    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        width, height = image.shape[:2]
        # Make Detections
        results = mp_pose.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        landmarks = []
        temp=[]
        if results.pose_landmarks:
        
            mp_drawing.draw_landmarks(image=image,landmark_list=results.pose_landmarks,connections=mpPose.POSE_CONNECTIONS)
            for landmark in results.pose_landmarks.landmark:
                    landmarks.append((int(landmark.x * width), int(landmark.y * height),(landmark.z * width)))
            left_elbow_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mpPose.PoseLandmark.LEFT_WRIST.value])

            # Get the angle between the right shoulder, elbow and wrist points. 
            right_elbow_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_WRIST.value])   
                    
            # Get the angle between the left elbow, shoulder and hip points. 
            left_shoulder_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value],
                                                landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                                landmarks[mpPose.PoseLandmark.LEFT_HIP.value])

            # Get the angle between the right hip, shoulder and elbow points. 
            right_shoulder_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value])

            # Get the angle between the left hip, knee and ankle points. 
            left_knee_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mpPose.PoseLandmark.LEFT_KNEE.value],
                                            landmarks[mpPose.PoseLandmark.LEFT_ANKLE.value])

            # Get the angle between the right hip, knee and ankle points 
            right_knee_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_ANKLE.value])
            
            
            left_hip_angle=calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                            landmarks[mpPose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mpPose.PoseLandmark.LEFT_KNEE.value])

            right_hip_angle=calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                            landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value])

            temp=[left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle,left_hip_angle,right_hip_angle]
        # Export coordinates
        try:
            # Extract Pose landmarks
            # pose = results.pose_landmarks.landmark
            # pose_row = list(np.array([[landmark.x, landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())
            
            X = pd.DataFrame([temp])
            
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class) 
            print(body_language_prob)
            
            print(np.max(body_language_prob))
            # Display Class
            
            
            
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
           
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
             break
cap.release()
cv2.destroyAllWindows()